# Script Goals
I am trying to generate a synthetic data set of patient outcome/survival --- I am going to have 1 (or 2) variables that are associated with the "true" outcome.. in this case I will use AGE as the  true correlate, and then will create 49 other randomly distributed variables for the patient Data set

I will then generate the actual survival where patient's > 80 convert to AD in ~ 1 year,  patients 70-80 convert in ~ 2 years, and patient's < 60 convert in 10 years

Also need to try various situations where the number of patients per group is NOT balanced and see if/how that effects the model explosion

In [7]:
### Safoora Friendly Format is a CSV file
from scipy import load
import lifelines
import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.graph_objs as go
import random
import numpy as np
import pandas as pd
import datetime
plotly.offline.init_notebook_mode(connected=True)

In [9]:
sampleAgingSet = pd.read_csv("./survivalData.csv")
sampleAgingSet.head()

,Patient ID,Survival Time,Censored Status,Age(Feature_0),feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,patientID_867,15,1,82,1.277817,1.131151,0.445461,0.938439,0.735632,1.974018,...,0.936157,2.139038,0.264897,2.829890,1.028059,1.272085,2.551403,0.237913,2.784960,1.538780
1,patientID_646,21,1,54,1.532690,0.827452,1.085001,1.317138,0.110652,2.538604,...,1.577645,0.543637,2.877652,0.420858,2.154877,1.539021,1.709432,1.367143,0.719639,0.986608
2,patientID_559,108,1,54,0.914156,1.635885,0.169837,1.627782,2.594463,2.432860,...,2.481730,1.144361,1.759727,0.961235,1.511825,1.766929,2.022466,0.568927,0.711459,0.780265
3,patientID_743,1,1,77,2.245585,2.493867,0.351268,1.397774,2.273481,0.016673,...,0.270703,2.379196,2.838301,0.507005,1.063033,1.411715,2.054420,2.200892,1.352230,1.778473
4,patientID_236,89,0,71,1.721017,0.203914,2.463101,2.353142,1.229570,1.987825,...,1.400681,0.306261,2.779289,0.632617,1.900929,1.824993,2.778487,1.788818,1.286664,0.973639


In [17]:
### Generate trace data for the features to make sure we did the randomization right
traceData = []

for ftr in range(1,50):
    trace= go.Box(x=sampleAgingSet['feature_%d' % ftr], name="Feature_%d" % ftr)
    traceData.append(trace)   

In [18]:
plotly.offline.iplot({ "data": traceData, "layout": Layout(title="Feature Dist")})

In [27]:
### Generate trace data for the features to make sure we did the randomization right
trace= go.Scatter(x=sampleAgingSet['Age(Feature_0)'], y = sampleAgingSet['Survival Time'], mode='markers')
plotly.offline.iplot({ "data": [trace], "layout": Layout(title="Feature Dist")})

In [31]:
import pickle
import scipy.io as sio 
import survivalnet as sn

# Integrated models. 
# Defines model/dataset pairs.
ModelPaths = ['../agingResults/']
Models = ['final_model']
Data = ['../agingDataSet/myAgingMatrix.mat']

# Loads datasets and performs feature analysis.
for i, Path in enumerate(ModelPaths):

        # Loads normalized data.
        X = sio.loadmat(Data[i])

        # Extracts relevant values.
        Samples = X['Patients']
        Normalized = X['Integ_X'].astype('float32')
        #Raw = X['Integ_X_raw'].astype('float32')
        #Symbols = X['Integ_Symbs']
        Survival = X['Survival']
        Censored = X['Censored']
        # Loads model.
        f = open(Path + Models[i], 'rb')
        Model = pickle.load(f)
        f.close()

        sn.analysis.FeatureAnalysis(Model, Normalized, Raw, Symbols,
                                                                Survival, Censored,
                                                                Tau=5e-2, Path=Path)


NameError: name 'Raw' is not defined

In [4]:
patientCohortSize = 1000
randVars = 50

## So have three labels,  old, older and oldest
### wrap it around np.array


# old is 60-70    may or may not convert, say 30% convert after 5 years
# older is 70-80  all develop disease   all develop disease within ~ 2 years
# oldest is 80-90  all develop disease within ~ 1 years

## let's make this reproducibly random!
random.seed(66)

AllPatientDictionary = {}

## Generate the old group
for pt in range(patientCohortSize):
    curPtDict = {}
    
    ptID  = "pt" + str(pt)
    AllPatientDictionary[ptID] = { "ptFeatures": {}, "ptLabel": "old", "censored": True/False,
                                  "survivalTime": }
    
    
    labels: { "group": "old"} ## NOTE THIS ISN"T REALLY NEEDED FOR THE MODEL, maybe this is just
            ## a descriptor for how the data was generated
    outcomes: { "survivalTime": "", "censored": True}
    
    if randint < 0.7:
        censor = True
        survivalTime = 60 ## means they were followed for 60 months and didn't progress
    else:
        censor = False
        survivalTime = random.randint(30,60)
        
    
#     ###, 
#                                   "censored": False, "survivalTime": random.randint()} 

    ## Most young patients don't get the disease, so
    censor = True
    survivalTime = random.randint( 60,100) ## means we followed for 5 years up to 10
    
    ## survival time is in months
    
    curPtDict['age'] = 60 + random.randint(1,10)
    ## The random variables will be distributed between 1 and 100?
    ## May want to play aroudn with floats and 
    for v in range(randVars):
        rv = random.randint(1,100)
        curPtDict['rv'+str(v)] = rv
    print curPtDict


{'rv17': 51, 'rv16': 46, 'rv15': 73, 'rv14': 17, 'rv13': 37, 'rv12': 7, 'rv11': 79, 'rv10': 13, 'rv35': 80, 'rv34': 88, 'rv37': 32, 'rv36': 41, 'rv31': 20, 'rv30': 30, 'rv19': 85, 'rv18': 53, 'rv32': 93, 'rv39': 2, 'rv38': 41, 'rv43': 66, 'rv42': 42, 'rv22': 89, 'rv23': 71, 'rv20': 98, 'rv21': 38, 'rv26': 21, 'rv27': 32, 'rv24': 70, 'rv25': 98, 'rv40': 12, 'rv41': 48, 'rv28': 49, 'rv29': 31, 'rv44': 57, 'rv45': 10, 'rv46': 85, 'rv47': 43, 'rv3': 26, 'rv2': 45, 'rv1': 25, 'rv0': 44, 'rv7': 93, 'rv6': 75, 'rv5': 57, 'rv4': 87, 'rv9': 47, 'rv8': 50, 'rv49': 38, 'rv48': 48, 'age': 61, 'ptID': 'pt0', 'rv33': 98}


NameError: name 'sys' is not defined